Use this file to clean the raw data and save the data to csv file

In [1]:
import csv
import re
import pandas as pd
import pickle
import torch

In [2]:
def clean(text):

    text = re.sub(r'http\S+','',text)
    text = re.sub(r'@\S*','',text)
    text = re.sub(r'\s+',' ',text)
    return text

In [3]:
training_data_name = ['twitter-2013dev-A.tsv', 'twitter-2013test-A.tsv', 'twitter-2013train-A.tsv',
                     'twitter-2014sarcasm-A.tsv', 'twitter-2014test-A.tsv', 'twitter-2015test-A.tsv',
                     'twitter-2015train-A.tsv', 'twitter-2016dev-A.tsv', 'twitter-2016devtest-A.tsv',
                     'twitter-2016test-A.tsv', 'twitter-2016train-A.tsv']

In [4]:
training_data = []
for name in training_data_name:

    with open('./raw_data/'+name) as f:
        reader = csv.reader(f, delimiter='\t')
        for line in reader:
            if len(line) < 3:
                continue
            sent = clean(line[2])
            if sent is not None: 
                training_data.append([line[1], sent])

In [5]:
with open('./modified_data/training_data.csv','w',newline='')as f:
    ff=csv.writer(f)
    ff.writerows(training_data)

In [6]:
testing_data = []

with open('./raw_data/SemEval2017-task4-test.subtask-A.english.txt') as f:
    for line in f:
        s = line.strip().split('\t')
        if len(s) < 3:
            continue
        sent = clean(s[2])
        if sent is not None: 
            testing_data.append([s[1], sent])

In [7]:
with open('./modified_data/testing_data.csv','w',newline='')as f:
    ff=csv.writer(f)
    ff.writerows(testing_data)

In [8]:
kaggle_data = pd.read_csv("./raw_data/training.1600000.processed.noemoticon.csv",header=None,encoding = "latin-1")

In [9]:
kaggle_data = kaggle_data[[0,5]]
kaggle_data[5] = kaggle_data[5].astype('str')
kaggle_data[5]=kaggle_data[5].replace(r'@\S*', '', regex=True).replace(r'http\S+', '', regex=True).replace(r'\s+', ' ', regex=True)
kaggle_data = kaggle_data.loc[kaggle_data[5].str.len() >= 24]

In [10]:
kaggle_data.to_csv('./modified_data/kaggle_data.csv', index=False, header=False)

In [11]:
word_to_ix = dict()
word_embeddings = torch.empty([658125,300],dtype=torch.float32)
with open('./raw_data/datastories.twitter.300d.txt') as f:
    for i, line in enumerate(f):
        line = line.strip().split()
        word_to_ix[line[0]] = len(word_to_ix)
        word_vec = line[1:]
        word_vec = [float(num) for num in word_vec]
        word_embeddings[i] = torch.FloatTensor(word_vec)

In [12]:
pickle.dump(word_to_ix,open('./modified_data/word_to_idx.pkl','wb'))
pickle.dump(word_embeddings,open('./modified_data/word_embeddings.pkl','wb'))